In [ ]:
# Pkg.build("Plots") 
using Plots
using Colors

# Set the default plot size to something that fits a cell
default(size = (800, 300))

In [ ]:
spread_propability = 0.3
death_probability = 0.3

In [ ]:
@enum VegetationType forest field knotweed

mutable struct Vegetation
    status::VegetationType
end


function create_map()
    cells = Array{Vegetation}(undef, 16, 16)
    for i in 1:size(cells)[1]
        for j in 1:size(cells)[2]
            cells[i,j] = Vegetation(field)
        end
    end
    for i in 1:size(cells)[1]
        cells[i,1].status = forest
    end
    cells[1,2].status = forest
    cells[2,2].status = forest
    cells[1,3].status = forest
    cells[2,3].status = knotweed
    return cells
end

# Map the cells to colors
function to_colors(cell::Vegetation)
    if cell.status == forest
        return RGB(0.0,0.8,0.0)
    end
    if cell.status == field
        return RGB(0.5,1.0,0.5)
    end
    if cell.status == knotweed
        return RGB(0.0,0.0,0.0)
    end
end

function interact!(new_cell::Vegetation, other_cell::Vegetation)
    if new_cell.status == field && other_cell.status == knotweed
        if rand(1)[1] < spread_propability
            new_cell.status = knotweed
        end
    end
end

function update!(new_cell::Vegetation)
    if new_cell.status == knotweed
        if rand(1)[1] < death_probability
            new_cell.status = field
        end
    end
end

function update(cells::Matrix{Vegetation})
    new_cells = deepcopy(cells)
    for i in 1:size(cells)[1]
        for j in 1:size(cells)[2]
            update!(new_cells[i,j])
        end
    end
    for i in 1:size(cells)[1]-1
        for j in 1:size(cells)[2]
            interact!(new_cells[i,j], cells[i+1,j])
            interact!(new_cells[i+1,j], cells[i,j])
        end
    end
    for i in 1:size(cells)[1]
        for j in 1:size(cells)[2]-1
            interact!(new_cells[i,j], cells[i,j+1])
            interact!(new_cells[i,j+1], cells[i,j])
        end
    end
    return new_cells
end


In [ ]:
import Plots: plot

function plot(cells::Matrix{Vegetation})
    plot(to_colors.(cells), legend=false, border=:none)
end

cells = create_map()
plot(cells)

In [ ]:
cells = create_map()

anim = @animate for i ∈ 1:20
    if i > 1
        cells = update(cells)
    end
    plot(to_colors.(cells), legend=false, border=:none)
end
gif(anim, "anim_fps15.gif", fps = 2)